In [0]:
%pip install psycopg2
%pip install sqlalchemy

In [0]:
 %restart_python

In [0]:
#Let's import needed libraries and classes
import requests
import json
from cryptography.fernet import Fernet
import pandas as pd
from pyspark.sql.functions import length, col
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import create_engine

In [0]:
#Let's read the config file 
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

In [0]:
# Import the function from the Python file
from process_urls import process_urls

# Get the list of URLs from the config and modify the URL format
urls = [url.replace("/tracker/mileage_logs", "/internal/deals") for url in config['track_milage_logs_urls']]
# Process the URLs to get a list of DataFrames
dataframes = process_urls(urls)

def merge_dataframes(dataframes):
    if dataframes:
        # Concatenate all DataFrames in the list into a single DataFrame
        final_dataframe = pd.concat(dataframes)
        return final_dataframe
    else:
        print("No DataFrames to concatenate")
        return pd.DataFrame()

# Merge the list of DataFrames into a single DataFrame
final_dataframe = merge_dataframes(dataframes)

To take into account in the final analysis, I was not able to load the data for the last month of 2024 due to a server error. I have tried to load partial data for the month but still got the same error.

In [0]:
final_dataframe

We will remove duplicate values



In [0]:
No_duplicate_data = final_dataframe.drop_duplicates()
No_duplicate_data.count()

In [0]:
No_duplicate_data.isnull().sum()

In [0]:
No_duplicate_data.rename(columns={'_id':'id', '_stage_n':'stage_n', '_stage':'stage', '_create_date':'create_date', '_is_deleted':'is_deleted'}, inplace=True)

In [0]:
# as missing values in only once record we will remove it 
No_duplicate_data = No_duplicate_data.dropna()

In [0]:

# Read configuration from the JSON file
with open('config_db.json', 'r') as config_file:
    config = json.load(config_file)

dbname = config["dbname"]
user = config["user"]
password = config["password"]
host = config["host"]
port = config["port"]

In [0]:

df = pd.DataFrame(No_duplicate_data)

# Create a SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')

# Upload the dataframe to the PostgreSQL database as a table
table_name = 'Internal_Deals'
df.to_sql(table_name, engine, index=False, if_exists='replace')